## 如何调试梯度

In [1]:
import numpy as np
import matplotlib.pyplot as plt

### 1. 模拟数据

In [12]:
np.random.seed(666)
X = np.random.random(size=(1000, 10)) # 有1000个样本，每个样本有10个特征

In [13]:
true_theta = np.arange(1, 12, dtype=float) #随机生成11个θ值
true_theta

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])

In [14]:
X_b = np.hstack([np.ones((len(X), 1)), X])
y = X_b.dot(true_theta) + np.random.normal(size=1000)

In [15]:
print(X.shape)
print(y.shape)

(1000, 10)
(1000,)


### 2. 定义损失函数 J

In [16]:
def J(theta, X_b, y):
    try:
        return np.sum((y - X_b.dot(theta)) ** 2) / len(X_b)
    except:
        return float('inf')

### 3. 定义导数

In [17]:
def dJ_math(theta, X_b, y): # 使用数学公式计算导数
    return X_b.T.dot(X_b.dot(theta) - y) * 2 / len(y)

def dJ_debug(theta, X_b, y, epsilon=0.01): # 使用调试的方法求导数
    res = np.empty(len(theta))
    for i in range(len(theta)):
        theta_1 = theta.copy()
        theta_1[i] += epsilon
        theta_2 = theta.copy()
        theta_2[i] -= epsilon
        res[i] = (J(theta_1, X_b, y) - J(theta_2, X_b, y)) / (2 * epsilon)
    return res

### 3. 使用梯度下降法进行验证

这里使用批量梯度下降法

In [20]:
def gradient_descent(dJ, X_b, y, initial_theta, eta, n_iters=1e4, epsilon=1e-8):
    theta = initial_theta
    i_iter = 0
    
    while i_iter < n_iters:
        gradient = dJ(theta, X_b, y)
        last_theta = theta
        theta =  theta - eta * gradient
        
        if(abs(J(theta, X_b, y) - J(last_theta, X_b, y)) < epsilon):
            break
        i_iter += 1
    return theta

In [22]:
# 使用 dJ_debug
X_b = np.hstack([np.ones((len(X), 1)), X])
initial_theta = np.zeros(X_b.shape[1])
eta = 0.01

%time theta = gradient_descent(dJ_debug, X_b, y, initial_theta, eta)
theta

Wall time: 2.6 s


array([ 1.1251597 ,  2.05312521,  2.91522497,  4.11895968,  5.05002117,
        5.90494046,  6.97383745,  8.00088367,  8.86213468,  9.98608331,
       10.90529198])

theta大概的结果就是1，2，3，4...11 这样的数，和我们模拟的 true_theta 是吻合的。

In [23]:
# 使用 dJ_math
%time theta = gradient_descent(dJ_math, X_b, y, initial_theta, eta)
theta

Wall time: 439 ms


array([ 1.1251597 ,  2.05312521,  2.91522497,  4.11895968,  5.05002117,
        5.90494046,  6.97383745,  8.00088367,  8.86213468,  9.98608331,
       10.90529198])

可以看到使用数学公式速度快了很多，而且结果也比较准确。